In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import os
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [2]:
from chat_memory import InSessionMemoryHistory, ChatHistory
from datetime import datetime
import uuid
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
MODEL_NAME = "llama3.2"
llm = OllamaLLM(model= MODEL_NAME)

In [4]:
def load_docs():
    
    document_loader = []

    for root, dirs, files in os.walk("."):
        # Skip chroma_db folder
        if "faiss" in root or "git" in root:
            continue
        for file in files:
            if file.endswith(".pdf"):
                document_loader.append(file)

    return document_loader

In [5]:
document_loader = load_docs()
document_loader

['ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf',
 'Electric Machinery Fundamentals Textbook -- Chapman.pdf',
 'ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf',
 'Three Phase Power System Fundamentals.pdf']

In [6]:
embedding_model ="sentence-transformers/all-MiniLM-L6-v2" #embedding matrix model

def embed_splitting(document_loader, embedding_model):
    embeddings = HuggingFaceEmbeddings(model = embedding_model, encode_kwargs={'normalize_embeddings': True})

    doc_store = []
    for file in document_loader:
        loader = PyPDFLoader(file)
        doc = loader.load()
        doc_store += doc

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 400,
        chunk_overlap = 64
        )
    
    #Make splits
    splits = text_splitter.split_documents(doc_store)

    return embeddings, splits


In [7]:
embeddings, splits = embed_splitting(document_loader, embedding_model)

In [8]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
len(splits)

402

In [10]:
dim = len(embeddings.embed_query("test sentence"))
index = faiss.IndexFlatL2(dim)

if os.path.exists("faiss_index"):
    print("Loading FAISS index from disk...")
    vector_store = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
else:
    print("Building FAISS index from scratch...")
    dim = len(embeddings.embed_query("test sentence"))
    index = faiss.IndexFlatL2(dim)
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(splits)
    vector_store.save_local("faiss_index")

Loading FAISS index from disk...


In [11]:
# create the retriever object once
semantic_retriever = vector_store.as_retriever(search_kwargs={'k': 4})

In [12]:
semantic_retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1085b5610>, search_kwargs={'k': 4})

In [13]:
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 4

In [14]:
ensemble_retriever = EnsembleRetriever(retrievers= [semantic_retriever, bm25_retriever], weights = [0.67, 0.33], search_kwargs={"k": 3})

In [15]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, ensemble_retriever, contextualize_q_prompt
)

In [16]:
user_input = "Explain transformers"
history_results = history_aware_retriever.invoke({"input": user_input, "chat_history": []})

In [17]:
for i, doc in enumerate(history_results):
    print(f"\nDocument {i+1}")
    print(f"Source: {doc.metadata.get('source', 'unknown')}")
    print(f"Content:\n{doc.page_content}")


Document 1
Source: ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf
Content:
Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shell-type construction. 
 
 
Figure 6-6  Core-type transformer 

In [18]:
user_input_prompt = """You are an expert assistant answering based only on the provided context.

    Context:
    {context}
    
    Use all relevant information above to answer the question below. If the answer isn't found in the chunks, say:
    "I cannot answer this question because the necessary information was not found in the provided documents."

    When answering, cite the **source file name** and **slide/page number** if available.
    """

In [19]:
prompt_template = ChatPromptTemplate(
    [
        ("system", user_input_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [20]:
question_answer_chain = create_stuff_documents_chain(llm, prompt_template)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
history = ChatHistory()

History table successfully created


In [22]:
### Statefully manage chat history ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InSessionMemoryHistory(session_id, db=history)
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [23]:
def pipeline_combined():
    session_id = str(uuid.uuid4())[:8]
    print(f"\nSession ID: {session_id}")

    print(f"\nModel {MODEL_NAME} has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.")
    try:
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Session ended. Have a good day.")
                break

            response = conversational_rag_chain.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}},
            )
            print(f"LLM: {response['answer']}\n")

            # Note: The memory is managed by the chain via get_session_history
            # So you don't need to manually add messages here
    finally:
        history.close()

In [24]:
pipeline_combined()


Session ID: 80d2313e

Model llama3.2 has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.


Error in RootListenersTracer.on_chain_end callback: ProgrammingError("Error binding parameter 4: type 'list' is not supported")


[DEBUG] Inserting message:
  session_id: <class 'str'> 80d2313e
  role: <class 'str'> human
  turn_number: <class 'int'> 1
  message: <class 'list'> [HumanMessage(content='Explain transformers', additional_kwargs={}, response_metadata={}), AIMessage(content='I can provide an explanation of transformers based on the provided context.\n\nA transformer is a practical application of magnetically coupled coils. It transfers energy from one coil to another by using a common magnetic core to improve coupling. The primary purpose of a transformer is to transfer energy while providing electrical isolation between the source and load, as well as changing voltage and current levels.\n\nThe basic principle of a transformer can be understood by considering two coils wound on opposite sides of a magnetic core, as shown in figure 6-6. When a sinusoidal voltage source drives the primary coil, it produces a sinusoidal magnetic flux that passes through both coils. This results in an induced voltage in t

Error in RootListenersTracer.on_chain_end callback: ProgrammingError("Error binding parameter 4: type 'list' is not supported")


[DEBUG] Inserting message:
  session_id: <class 'str'> 80d2313e
  role: <class 'str'> human
  turn_number: <class 'int'> 2
  message: <class 'list'> [HumanMessage(content='What materials are they made of?', additional_kwargs={}, response_metadata={}), AIMessage(content='The cores for high-frequency transformers are often made from magnetically soft ferrites, while power frequency transformer cores are typically made from an iron alloy such as silicon steel.\n\nAdditionally, small power transformers are commonly made with a shell-type construction, where the coils are wound on a laminated core. The center limb is twice the width of the outer limbs because it carries twice the flux.\n\nIn induction machines, the rotor conductors are placed in slots in a laminated steel core, and are made from materials such as aluminium bars, which are connected to short-circuiting aluminium rings known as end-rings.\n\nNo other information about specific materials is mentioned.', additional_kwargs={}, r

In [25]:
conversational_rag_chain.invoke(
    {"input": "Explain transformers?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

ProgrammingError: Cannot operate on a closed database.

In [ ]:
print(get_session_history("abc123").messages)